In [1]:
import pandas as pd 
import deepmatcher as dm

In [2]:
hybrid_mod = dm.MatchingModel(attr_summarizer ='hybrid')
hybrid_mod.load_state('../../models/beer_hybrid.pth')

In [3]:
test = dm.data.process_unlabeled(path='../../Structured/Beer/positives.csv',trained_model=hybrid_mod,
                                 ignore_columns=['id','label'])

In [4]:
predictions = hybrid_mod.run_prediction(test,output_attributes=True)
pos_predictions = predictions[predictions['match_score']>0.5]

===>  PREDICT Epoch 6
Finished Epoch 6 || Run Time:    0.4 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [5]:
pos_predictions.shape[0]

64

In [15]:
## Alter test set
test_df = pd.read_csv('../../Structured/Beer/positives.csv')
test_df ['ltable_Beer_Name'] = 'Flying Monkeys Amber Ale'
test_df ['rtable_Beer_Name'] = 'Flying Monkey Amber Ale'
test_df.to_csv('alt_test.csv',index=False)

In [16]:
alt_test = dm.data.process_unlabeled(path='alt_test.csv',trained_model=hybrid_mod,
                                 ignore_columns=['id','label'])

In [17]:
predictions = hybrid_mod.run_prediction(alt_test,output_attributes=True)
pos_predictions = predictions[predictions['match_score']>0.5]

===>  PREDICT Epoch 6
Finished Epoch 6 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [18]:
pos_predictions.shape[0]

54